In [1]:
from main import *

In [2]:
panel_properties_file = "../reflectance_panel_example_data.json"
geopackage_file = "../data/20240529_uav_multispectral_orthos_20m/20240529_tarps_locations.gpkg"
orthophotos_dir = "../data/20240529_uav_multispectral_orthos_20m/orthophotos"

In [3]:
panels = load_panel_properties(panel_properties_file)
panel_locations = load_panel_locations(geopackage_file)
orthophoto_paths = get_orthophoto_paths(orthophotos_dir)
with rasterio.open(orthophoto_paths[0]) as orthophoto:
    number_of_bands = len(orthophoto.read())
assert len(panels) == len(panel_locations)

# 1. For each image extract each panel intensities for each band

In [4]:
intensities = np.zeros((len(orthophoto_paths), len(panel_locations), number_of_bands))
intensities.shape

(50, 3, 6)

In [5]:
orthophoto_paths = list(sorted(orthophoto_paths))

In [6]:
# Extract intensities
for photo_index, orthophoto_path in enumerate(orthophoto_paths):
    with rasterio.open(orthophoto_path) as orthophoto:
        photo = orthophoto.read()
    for panel_index, panel in enumerate(panel_locations):
        if not is_panel_in_orthophoto(orthophoto_path, panel):
            intensities[photo_index][panel_index] = np.full(len(photo), np.NaN)
            continue
        for band_index, band in enumerate(photo):
            panel_intensity = extract(band, panel)
            intensities[orthophoto_path][panel_index][band_index] = panel_intensity
intensities

array([[[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, 

In [7]:
for panel_index, panel in enumerate(panel_locations):
    for band_index, band in enumerate(photo):
        intensities[:, panel_index, band_index] = interpolate(intensities[:, panel_index, band_index])
intensities

No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.
No values found for interpolation.


array([[[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, 

(50, 6)

In [ ]:
unconvertable_photos = []
for photo_index, orthophoto_path in enumerate(orthophoto_paths):
    for band_index, band in enumerate(photo):
        intensities_of_panels = intensities[photo_index, :, band_index]
        if np.isnan(intensities_of_panels).any():
            unconvertable_photos.append((photo_index, band_index))
            continue
        coeffs = fit(intensities_of_panels, get_band_reflectance(panels, band_index))
        convert(orthophoto_path, coeffs)